In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
def teste_cp(n_cp, Δx):
    f = lambda x: (x**(n_cp+1))*np.exp(0.75*x)
#     f = lambda x: np.exp(-20*x**2)
#     f = lambda x: -np.sin(np.pi*x)-0.5*x**3 + np.where(x > 0, 1, 0)
    x = np.arange(-1-3*Δx, 1+3*Δx, Δx, dtype = float_pres)
    
    return f(x)

def ref_cp(n_cp, Δx):
    df = lambda x: (n_cp+1)*(x**n_cp)*np.exp(0.75*x) + 0.75*(x**(n_cp+1))*np.exp(0.75*x)
#     df = lambda x: -40*x*np.exp(-20*x**2)
#     df = lambda x: -np.pi*np.cos(np.pi*x)-1.5*x**2
    x  = np.arange(-1-3*Δx, 1+3*Δx, Δx, dtype = float_pres)
    
    return df(x)

In [3]:
nomes = [
    "WENO-JS"     , "WENO-Z"      , "WENO-Z+"    , 
    "WENO-JS (M)" , "WENO-Z (M)"  , "WENO-Z+ (M)" , 
    "WENO-JS (MS)", "WENO-Z (MS)" , "WENO-Z+ (MS)", 
    "WENO-JS (BI)", "WENO-Z (BI)" , "WENO-Z+ (BI)",
    "WENO-ZC (M)" , "WENO-ZC (MS)", "WENO-ZC (BI)"
]

API      = API_Numpy
equation = diff_equation

WENOs = {}
WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS, mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z , mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 2]] = simulation(API, equation, WENO_Zp, mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 3]] = simulation(API, equation, WENO_JS, mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 4]] = simulation(API, equation, WENO_Z , mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 5]] = simulation(API, equation, WENO_Zp, mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 6]] = simulation(API, equation, WENO_JS, mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 7]] = simulation(API, equation, WENO_Z , mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 8]] = simulation(API, equation, WENO_Zp, mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 9]] = simulation(API, equation, WENO_JS, mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[10]] = simulation(API, equation, WENO_Z , mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[11]] = simulation(API, equation, WENO_Zp, mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[12]] = simulation(API, equation, WENO_ZC, mapping = post_inv_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[13]] = simulation(API, equation, WENO_ZC, mapping =  pre_inv_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[14]] = simulation(API, equation, WENO_ZC, mapping =  pre_inv_mapping, map_function = BI_mapping).DerivadaEspacial

In [14]:
base = 0.5
Δxs  = 2/20*base**np.arange(0, 10+1)
# Δxs  = 2/160*base**np.arange(0, 10+1)
# malha = [2/160, 2/320, 2/640, 2/1280, 2/2560, 2/5120, 2/10240, 2/20480]

fronteira = FronteiraFixa
n_cp = 0

testes    = {}
derivadas = {}
erros     = {}
for nome in nomes:
    
    resultados = []
    derivada   = []
    erro       = []
    for Δx in Δxs:
        u     = teste_cp(n_cp, Δx)
        u_hat = WENOs[nome](u, Δx, fronteira)
        dif   = u_hat - ref_cp(n_cp, Δx)
        
        resultados.append(np.max(np.abs(dif[3:-3])))             # norma L-inf
#         resultados.append(np.sqrt(np.sum(np.square(dif[3:-3])))) # norma L2
#         resultados.append((np.sum(np.abs(dif[3:-3])))            # norma L1
        
        derivada.append(u_hat)
        erro.append(dif[3:-3])
        
    testes[nome]    = resultados
    derivadas[nome] = derivada
    erros[nome]     = erro

In [7]:
derivadas["WENO-Z"][0].dtype

dtype('float64')

In [15]:
aux_nome = "WENO-Z"

aux_x = Δxs
aux_y = derivadas[aux_nome]

x     = tf.range(-1-3*aux_x[0], 1+3*aux_x[0], aux_x[0], dtype = float_pres)
y     = aux_y[0]
label = tf.repeat([2/Δxs[0]], [len(aux_y[0])])

for i in tf.range(1, len(aux_x)):
    x = tf.concat([x, tf.range(-1-3*aux_x[i], 1+3*aux_x[i], aux_x[i], dtype = float_pres)], axis = 0)
    y = tf.concat([y, aux_y[i]], axis = 0)
    label = tf.concat([label, tf.repeat([2/Δxs[i]], [len(aux_y[i])])], axis = 0)

# x     = tf.range(-1, 1, aux_x[0], dtype = float_pres)
# y     = aux_y[0]
# label = tf.repeat([2/Δxs[0]], [len(aux_y[0])])

# for i in tf.range(1, len(aux_x)):
#     x = tf.concat([x, tf.range(-1, 1, aux_x[i], dtype = float_pres)], axis = 0)
#     y = tf.concat([y, aux_y[i]], axis = 0)
#     label = tf.concat([label, tf.repeat([2/Δxs[i]], [len(aux_y[i])])], axis = 0)

    
df = pd.DataFrame(
    {
        "x"     : x     ,
        "y"     : y     ,
        "label" : label
    }
)

In [ ]:
fig = ply_exp.line(
    data_frame = df    ,
    x          = x     ,
    y          = y     ,
    color      = label
)
fig.add_trace(
    ply_go.Scatter(
        x    = tf.range(-1-3*aux_x[len(Δxs)-1], 1+3*aux_x[len(Δxs)-1], aux_x[len(Δxs)-1], dtype = float_pres),
        y    = ref_cp(n_cp, aux_x[len(Δxs)-1]) , # u_ref, 
        mode = "lines"                                           ,
        line = ply_go.scatter.Line(color="#666666")              ,
        name = "Referência"
    )
)
fig.show()

In [ ]:
fig.write_html("C:/Users/daniw/OneDrive/Documentos/GitHub/Top_mat_A/WENO-BI/ordem_de_convergência_gausiana.html")

In [16]:
testes

{'WENO-JS': [8.312136566823725e-07,
  2.1231544189426188e-08,
  4.033419709337238e-10,
  6.630418436515129e-12,
  1.2390088954816747e-13,
  3.0020430585864233e-13,
  5.857536677922326e-13,
  1.340261235327489e-12,
  3.0828672947791347e-12,
  6.358913395843047e-12,
  1.3610890192694569e-11],
 'WENO-Z': [9.121371147280399e-08,
  3.1253755139459827e-10,
  4.9613646524448995e-12,
  9.370282327836321e-14,
  1.474376176702208e-13,
  5.559996907322784e-13,
  6.323830348264892e-13,
  1.3020695632803836e-12,
  2.9665159217984183e-12,
  7.188916129052814e-12,
  1.3780088181647443e-11],
 'WENO-Z+': [3.5062689351939014e-07,
  5.059059948742117e-09,
  1.7816842445839143e-10,
  3.2150393458607596e-12,
  1.4832579608992091e-13,
  6.981082378842984e-13,
  6.754596881819452e-13,
  1.5214496329463145e-12,
  3.169464690699897e-12,
  7.810641022842901e-12,
  1.2863488052516914e-11],
 'WENO-JS (M)': [1.920676684363798e-08,
  3.1254643317879527e-10,
  4.9613646524448995e-12,
  1.0169642905566434e-13,
  1.12

In [17]:
ordem = {}
for nome in nomes:
    l_erro = np.log(testes[nome])/np.log(base)
    l_dif  = l_erro[1:]-l_erro[:-1]
    ordem[nome] = l_dif

In [18]:
ordem

{'WENO-JS': array([ 5.29093815,  5.71806196,  5.92675981,  5.74184147, -1.27675812,
        -0.96434941, -1.19414824, -1.20175856, -1.04450747, -1.09790926]),
 'WENO-Z': array([ 8.18907736,  5.97714873,  5.72650067, -0.65394024, -1.91497942,
        -0.18571458, -1.04193596, -1.187963  , -1.27700475, -0.93873894]),
 'WENO-Z+': array([ 6.11492351,  4.82755575,  5.79226148,  4.43799496, -2.23468121,
         0.04758106, -1.171505  , -1.05879262, -1.30120176, -0.71976903]),
 'WENO-JS (M)': array([ 5.94140034,  5.97718973,  5.60839607, -0.1494809 , -2.30133416,
        -0.51894533, -1.23132555, -0.50032269, -1.44202473, -1.07501621]),
 'WENO-Z (M)': array([ 5.94021166,  5.97708723,  5.72650067, -0.81805962, -1.75086004,
         0.06120016, -1.81147103, -0.69738014, -0.92966348, -1.25404273]),
 'WENO-Z+ (M)': array([ 5.94002619,  5.97708723,  5.18550501, -0.11728352, -1.91064048,
        -0.1714606 , -1.57881027, -0.66534267, -1.00345143, -1.56889367]),
 'WENO-JS (MS)': array([ 7.36011866,